In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr, spearmanr

In [2]:
ls *.csv

All_FieldData_Masud_Rrs.csv
All_FieldData_Masud_Rrs_rm_out400_800.csv
All_FieldData_Masud_Rrs_rm_out.csv
all_TSS_algorithms.csv
DatA_Q2_Sav_Altamah.csv
GLORIA_plus_lab_SAB_400nm_800nm_all_python.csv
GLORIA_Rrs_flagged_4_analysis.csv
GLORIA_Rrs_flagged_without_flag_info.csv
GLORIA_Rrs_Nov12_400-800nm_Nans_TSS_GT_500_flagged_out_PACE_SRF.csv
GLORIA_Rrs_Nov12_400-800nm_Nans_TSS_GT_500_flagged_out_PACE_SRF_wvl.csv
LabDataTestRSR.csv
LabDataTestRSR_rrs.csv
LabDataTestRSR_wvl.csv
Modis_aqua.csv
MODIS_rrs_tss.csv
OLCI_rrs_tss.csv
SOLID_Modis_Rrs_analysis_cloud_20.csv
SOLID_Modis_Rrs_analysis_cloud_20_labonly.csv
SOLID_MODIS_tss.csv
TSS_comparison_SOLID.csv
tss_mm.csv
tss_OLCI.csv
TSS_OLCI_Matchups.csv


In [3]:
# Load your CSV file
#df = pd.read_csv('GLORIA_plus_lab_SAB_400nm_800nm_all_python.csv')
df = pd.read_csv('/home/jovyan/TSS_ML/04Sept25/Filtered_Subset_GLORIA_Rrs_all_Vars_350_900_TSS.csv')
# Print the column names
print(df.columns)

Index(['GID', 'Latitude', 'Longitude', 'Date_Time_UTC', 'Cloud_fraction',
       'Skyglint_removal', 'Bias_removal_in_NIR', 'Self_shading_correction',
       'Chla_plus_phaeo', 'aCDOM440',
       ...
       'Rrs_891', 'Rrs_892', 'Rrs_893', 'Rrs_894', 'Rrs_895', 'Rrs_896',
       'Rrs_897', 'Rrs_898', 'Rrs_899', 'Rrs_900'],
      dtype='object', length=571)


In [5]:
# Read the 'Age' column
Rrs_668 = df['Rrs_668']
Rrs_665 = df['Rrs_665']
Rrs_750 = df['Rrs_750']
Rrs_555 = df['Rrs_555']
Rrs_667 = df['Rrs_667']
Rrs_560 = df['Rrs_560']
Rrs_470 = df['Rrs_470']
Rrs_488 = df['Rrs_488']
Rrs_645 = df['Rrs_645']


In [1]:
!ls *.csv

All_FieldData_Masud_Rrs.csv			SOLID_MODIS_tss.csv
All_FieldData_Masud_Rrs_rm_out.csv		TSS_comparison_SOLID.csv
GLORIA_plus_lab_SAB_400nm_800nm_all_python.csv	tss_mm.csv
MODIS_rrs_tss.csv				tss_OLCI.csv
OLCI_rrs_tss.csv


In [18]:
# Nechad et al 2009
# Set default values for these parameters
a =  355.85
b =  1.74
c =  1728
Nechad = 1.74 + (355.85 * np.pi * Rrs_665) / (1 - (np.pi * Rrs_665) / 1728)

In [19]:
# MillerMckee
# Set default values for these parameters
a =  1140.25
b =  1.91
Miller  = a * Rrs_668 - b


In [20]:
# Petus et al.
# Set default values for these parameters
a = 12450
b = 666.1
c = 0.48
Petus = a * Rrs_668 ** 2 + b * Rrs_668 + c

In [21]:
# Xu et al. 2021
#  to get rrs, rrs = rho_green/pi or  rho_red/pi
rho_green = Rrs_555 * np.pi;
rho_red = Rrs_667 * np.pi;
Xu = 3793.7*((rho_green + rho_red)**2) -16.5; 

In [22]:

# Alvado et al, (2021), https://doi.org/10.3390/w13182453 TSM (mg/L)
# for rivers and coasts Rsq = 0.79 with insitu validation
Alvado = 293.06 * Rrs_750 + 1.5369; 

In [23]:
# Jiang and Liu (2011) 
Jiang = 1365.5 * ((Rrs_470 + Rrs_555)**2) - 369.08*(Rrs_470 + Rrs_555) + 27.216

In [24]:
# Zhang et al. (2010a,b)

log10TSS = 0.6311 + 22.2158*(Rrs_555 + Rrs_645) - 0.5239*(Rrs_488/Rrs_555)
Zhang = 10 ** log10TSS

In [25]:
# Hou et al 2017 model 1 - 300 mg/L TSS (http://doi.org/10.1016/j.rse.2016.12.006)
Hou = 207.19 * (Rrs_665/Rrs_560)**2 -114.78 * (Rrs_665/Rrs_560)

In [26]:
TSS = df['TSS']

In [27]:
!pwd


/home/jovyan/TSS_ML


In [28]:
# Combine the variables into a DataFrame
data = pd.DataFrame({
    'TSS': TSS,
    'Nechad': Nechad,
    'Hou': Hou,
    'Zhang': Zhang,
    'Alvado': Alvado,
    'Miller': Miller,
    'Xu': Xu,
    'Petus': Petus,
    'Jiang': Jiang
    
})

# Save the DataFrame as a CSV file
data.to_csv('TSS_global_filtered_data.csv', index=False)

In [ ]:

# Create a dictionary to store the variables
data = {
    'variable1': [1, 2, 3, 4, 5],
    'variable2': ['A', 'B', 'C', 'D', 'E'],
    # ... add more variables here ...
    'variable10': [10, 20, 30, 40, 50]
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Save the DataFrame as a CSV file
df.to_csv('combined_variables.csv', index=False)

In [ ]:


# Hu et al. (2004) MODIS 250-m, lower ranges 2-11 mg/l
# TSS_hu = 0.00522*(exp(1002*(Rrs_645 - Rrs_859))); # Tampa Bay, United States



# Cai et al. 2016 model 100-800 mg/L TSS (http://doi.org/10.1080/01431161.2015.1121302)
# TSS_cai = 673.15 * Rrs_665 + 8344.9 * Rrs_865 -11.219







In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.pipeline import Pipeline

In [10]:
# Step 1: Load the CSV file
# Replace 'your_file.csv' with the actual path to your CSV file
df = pd.read_csv('/home/jovyan/TSS_ML/11Aug25/SAB_QWIP_LT03_GLORIA_LAB_RSR_PACE.csv')

# Step 2: Rename the band columns to 'Rrs_XXX'
band_cols = [col for col in df.columns if col not in ['Latitude', 'Longitude', 'TSS']]
for col in band_cols:
    df.rename(columns={col: f'Rrs_{col}'}, inplace=True)

In [12]:
# Step 3: Define selected bands (focusing on specified wavelengths)
# Note: Assuming exact column names exist; if not, find closest wavelengths manually
selected_wavelengths = ['488', '555', '560', '665', '668', '695', '700', '710']
selected_cols = [f'Rrs_{w}' for w in selected_wavelengths]

# Check for missing columns
missing = [col for col in selected_cols if col not in df.columns]
if missing:
    print(f"Warning: Missing columns {missing}. Please check if exact wavelengths exist or use closest.")

# Additional columns needed for direct models
extra_cols = ['Rrs_645', 'Rrs_488', 'Rrs_555', 'Rrs_560', 'Rrs_665']
for col in extra_cols:
    if col not in df.columns:
        print(f"Warning: Missing {col} for direct models.")

In [13]:
# Step 4: Extract features and target
X_all = df[[col for col in df.columns if col.startswith('Rrs_')]]
X_selected = df[selected_cols]
y = df['TSS']

# Step 5: Split data into train (80%) and test (20%)
X_train_all, X_test_all, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=42)
X_train_sel, X_test_sel, _, _ = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Get test dataframe for direct models
test_indices = X_test_all.index
df_test = df.loc[test_indices]


In [ ]:
# Step 6: Implement direct relationship models and evaluate on test set

# Function to evaluate predictions
def evaluate(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} - MSE: {mse:.4f}, R²: {r2:.4f}")

# 6.1 Zhang et al. (2010a,b)
log10TSS_zhang = 0.6311 + 22.2158 * (df_test['Rrs_555'] + df_test['Rrs_645']) - 0.5239 * (df_test['Rrs_488'] / df_test['Rrs_555'])
zhang_pred = 10 ** log10TSS_zhang
evaluate(y_test, zhang_pred, 'Zhang')

# 6.2 Nechad et al. (2009)
a = 355.85
b = 1.74
c = 1728
nechad_pred = b + (a * np.pi * df_test['Rrs_665']) / (1 - (np.pi * df_test['Rrs_665']) / c)
evaluate(y_test, nechad_pred, 'Nechad')

# 6.3 Hou
ratio = df_test['Rrs_665'] / df_test['Rrs_560']
hou_pred = 207.19 * (ratio ** 2) - 114.78 * ratio
evaluate(y_test, hou_pred, 'Hou')

In [14]:

# Step 6: Implement direct relationship models and evaluate on test set

# Function to evaluate predictions
def evaluate(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} - MSE: {mse:.4f}, R²: {r2:.4f}")

# 6.1 Zhang et al. (2010a,b)
log10TSS_zhang = 0.6311 + 22.2158 * (df_test['Rrs_555'] + df_test['Rrs_645']) - 0.5239 * (df_test['Rrs_488'] / df_test['Rrs_555'])
zhang_pred = 10 ** log10TSS_zhang
evaluate(y_test, zhang_pred, 'Zhang')

# 6.2 Nechad et al. (2009)
a = 355.85
b = 1.74
c = 1728
nechad_pred = b + (a * np.pi * df_test['Rrs_665']) / (1 - (np.pi * df_test['Rrs_665']) / c)
evaluate(y_test, nechad_pred, 'Nechad')

# 6.3 Hou
ratio = df_test['Rrs_665'] / df_test['Rrs_560']
hou_pred = 207.19 * (ratio ** 2) - 114.78 * ratio
evaluate(y_test, hou_pred, 'Hou')

# Step 7: Preprocessing setups
scaler = StandardScaler()

# 7.1 Polynomial Features example (degree 2 on selected features)
poly = PolynomialFeatures(degree=2)
X_train_sel_poly = poly.fit_transform(X_train_sel)
X_test_sel_poly = poly.transform(X_test_sel)

X_train_sel_poly_scaled = scaler.fit_transform(X_train_sel_poly)
X_test_sel_poly_scaled = scaler.transform(X_test_sel_poly)

# Step 8: Feature Selection example
# 8.1 SelectKBest on all features
selector_kbest = SelectKBest(f_regression, k=10)  # Select top 10 features
X_train_all_kbest = selector_kbest.fit_transform(X_train_all, y_train)
X_test_all_kbest = selector_kbest.transform(X_test_all)

X_train_all_kbest_scaled = scaler.fit_transform(X_train_all_kbest)
X_test_all_kbest_scaled = scaler.transform(X_test_all_kbest)

# 8.2 RFE with LinearRegression on selected features
estimator = LinearRegression()
selector_rfe = RFE(estimator, n_features_to_select=5)  # Select 5 features
X_train_sel_rfe = selector_rfe.fit_transform(X_train_sel, y_train)
X_test_sel_rfe = selector_rfe.transform(X_test_sel)

X_train_sel_rfe_scaled = scaler.fit_transform(X_train_sel_rfe)
X_test_sel_rfe_scaled = scaler.transform(X_test_sel_rfe)

# Step 9: ML Models with Hyperparameter Tuning using GridSearchCV
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'SVR': SVR(),
    'GradientBoosting': GradientBoostingRegressor(random_state=42)
}

# Hyperparameter grids
param_grids = {
    'Ridge': {'alpha': [0.1, 1.0, 10.0]},
    'Lasso': {'alpha': [0.1, 1.0, 10.0]},
    'RandomForest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]},
    'SVR': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
    'GradientBoosting': {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1], 'max_depth': [3, 5]}
}

# Function to train and evaluate with optional grid search
def train_evaluate(model, X_train, X_test, y_train, y_test, name, use_grid=False, param_grid=None, scaled=False):
    if scaled:
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    if use_grid and param_grid:
        grid = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        print(f"Best params for {name}: {grid.best_params_}")
    else:
        best_model = model
        best_model.fit(X_train, y_train)
    
    pred = best_model.predict(X_test)
    evaluate(y_test, pred, name)

# Step 10: Run models on different feature sets

# 10.1 Using all Rrs bands (with tuning where applicable)
print("\nResults using all Rrs bands:")
for name, model in models.items():
    use_scaled = name in ['Ridge', 'Lasso', 'SVR']
    train_evaluate(model, X_train_all, X_test_all, y_train, y_test, f'{name}_all',
                   use_grid=(name in param_grids), param_grid=param_grids.get(name), scaled=use_scaled)

# 10.2 Using selected Rrs bands
print("\nResults using selected Rrs bands:")
for name, model in models.items():
    use_scaled = name in ['Ridge', 'Lasso', 'SVR']
    train_evaluate(model, X_train_sel, X_test_sel, y_train, y_test, f'{name}_selected',
                   use_grid=(name in param_grids), param_grid=param_grids.get(name), scaled=use_scaled)

# 10.3 Using polynomial features on selected
print("\nResults using polynomial features on selected:")
for name, model in models.items():
    use_scaled = True  # Polynomial features benefit from scaling
    train_evaluate(model, X_train_sel_poly, X_test_sel_poly, y_train, y_test, f'{name}_poly',
                   use_grid=(name in param_grids), param_grid=param_grids.get(name), scaled=use_scaled)

# 10.4 Using SelectKBest on all
print("\nResults using SelectKBest on all:")
for name, model in models.items():
    use_scaled = name in ['Ridge', 'Lasso', 'SVR']
    train_evaluate(model, X_train_all_kbest, X_test_all_kbest, y_train, y_test, f'{name}_kbest',
                   use_grid=(name in param_grids), param_grid=param_grids.get(name), scaled=use_scaled)

# 10.5 Using RFE on selected
print("\nResults using RFE on selected:")
for name, model in models.items():
    use_scaled = name in ['Ridge', 'Lasso', 'SVR']
    train_evaluate(model, X_train_sel_rfe, X_test_sel_rfe, y_train, y_test, f'{name}_rfe',
                   use_grid=(name in param_grids), param_grid=param_grids.get(name), scaled=use_scaled)

# Additional: You can combine in a pipeline, e.g.,
# pipeline = Pipeline([('poly', PolynomialFeatures(degree=2)), ('selector', SelectKBest(f_regression, k=10)), ('model', Ridge())])
# Then use GridSearchCV on the pipeline with parameters like {'model__alpha': [0.1,1,10]}

Zhang - MSE: 838.9329, R²: -0.3910
Nechad - MSE: 753.9593, R²: -0.2501
Hou - MSE: 10472.5553, R²: -16.3640

Results using all Rrs bands:
LinearRegression_all - MSE: 873.2592, R²: -0.4479
Best params for Ridge_all: {'alpha': 10.0}
Ridge_all - MSE: 539.7748, R²: 0.1050


/srv/conda/envs/notebook/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.542e+02, tolerance: 1.275e+01
  model = cd_fast.enet_coordinate_descent(
/srv/conda/envs/notebook/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.789e+01, tolerance: 1.228e+01
  model = cd_fast.enet_coordinate_descent(
/srv/conda/envs/notebook/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation.

Best params for Lasso_all: {'alpha': 0.1}
Lasso_all - MSE: 544.4972, R²: 0.0972
Best params for RandomForest_all: {'max_depth': 10, 'n_estimators': 200}
RandomForest_all - MSE: 601.7464, R²: 0.0023
Best params for SVR_all: {'C': 0.1, 'kernel': 'rbf'}
SVR_all - MSE: 604.9186, R²: -0.0030
Best params for GradientBoosting_all: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100}
GradientBoosting_all - MSE: 573.6776, R²: 0.0488

Results using selected Rrs bands:
LinearRegression_selected - MSE: 571.1994, R²: 0.0529
Best params for Ridge_selected: {'alpha': 0.1}
Ridge_selected - MSE: 584.3562, R²: 0.0311
Best params for Lasso_selected: {'alpha': 0.1}
Lasso_selected - MSE: 577.6901, R²: 0.0422
Best params for RandomForest_selected: {'max_depth': 10, 'n_estimators': 50}
RandomForest_selected - MSE: 639.4719, R²: -0.0603
Best params for SVR_selected: {'C': 0.1, 'kernel': 'rbf'}
SVR_selected - MSE: 605.9655, R²: -0.0047
Best params for GradientBoosting_selected: {'learning_rate': 0.01, 

/srv/conda/envs/notebook/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.485e+01, tolerance: 1.275e+01
  model = cd_fast.enet_coordinate_descent(


Best params for RandomForest_poly: {'max_depth': 20, 'n_estimators': 200}
RandomForest_poly - MSE: 651.6021, R²: -0.0804
Best params for SVR_poly: {'C': 0.1, 'kernel': 'rbf'}
SVR_poly - MSE: 605.2698, R²: -0.0036
Best params for GradientBoosting_poly: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100}
GradientBoosting_poly - MSE: 597.0364, R²: 0.0101

Results using SelectKBest on all:
LinearRegression_kbest - MSE: 592.8801, R²: 0.0170
Best params for Ridge_kbest: {'alpha': 0.1}
Ridge_kbest - MSE: 579.1305, R²: 0.0398
Best params for Lasso_kbest: {'alpha': 10.0}
Lasso_kbest - MSE: 603.1548, R²: -0.0001
Best params for RandomForest_kbest: {'max_depth': 10, 'n_estimators': 200}
RandomForest_kbest - MSE: 633.2704, R²: -0.0500
Best params for SVR_kbest: {'C': 0.1, 'kernel': 'rbf'}
SVR_kbest - MSE: 608.5489, R²: -0.0090
Best params for GradientBoosting_kbest: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}
GradientBoosting_kbest - MSE: 575.5455, R²: 0.0457

Results usin

In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from itertools import combinations


In [84]:

# Step 1: Load the CSV file
# Replace 'your_file.csv' with the actual path to your CSV file
df = pd.read_csv('/home/jovyan/TSS_ML/11Aug25/cleaned_QWIP_LT0.3_GLORIA_SAB_Rrs_11Aug25_400_799nm_Nans_TSS_GT_500_flagged_out_PACE_RSR.csv')

In [85]:
# First i have to run the Step1, otherwsie this will come with errros
# Step 2: Rename the band columns to 'Rrs_XXX' if not already
band_cols = [col for col in df.columns if col not in ['Latitude', 'Longitude', 'TSS']]
for col in band_cols:
    df.rename(columns={col: f'Rrs_{col}'}, inplace=True)

# Step 3: Get all available wavelengths as strings
all_wavelengths = sorted([col.split('_')[1] for col in df.columns if col.startswith('Rrs_')], key=float)

# Step 4: Define selected wavelengths (as strings to match column format)
selected_wl = ['488', '555', '560', '665', '668', '688']

# Step 5: Expand to include 2 bands before and after each selected band
expanded_wl = set()
for wl in selected_wl:
    if wl in all_wavelengths:
        i = all_wavelengths.index(wl)
        for j in range(-2, 3):  # -2, -1, 0, 1, 2
            idx = i + j
            if 0 <= idx < len(all_wavelengths):
                expanded_wl.add(all_wavelengths[idx])
    else:
        # Find closest wavelength if exact match is missing
        wl_float = float(wl)
        closest_wl = min(all_wavelengths, key=lambda x: abs(float(x) - wl_float))
        print(f"Warning: {wl} not found, using closest wavelength {closest_wl}")
        i = all_wavelengths.index(closest_wl)
        for j in range(-2, 3):
            idx = i + j
            if 0 <= idx < len(all_wavelengths):
                expanded_wl.add(all_wavelengths[idx])

# Convert to column names
expanded_cols = [f'Rrs_{wl}' for wl in sorted(expanded_wl, key=float)]

# Check for missing columns
missing = [col for col in expanded_cols if col not in df.columns]
if missing:
    print(f"Warning: Missing columns {missing}. Check DataFrame columns.")
    # Filter out missing columns
    expanded_cols = [col for col in expanded_cols if col in df.columns]

# Step 6: Extract features and target
X_expanded = df[expanded_cols]
y = df['TSS']

# Step 7: Split data into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(X_expanded, y, test_size=0.2, random_state=42)

In [86]:
# Step 8: Evaluation function
def evaluate(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} - MSE: {mse:.4f}, R²: {r2:.4f}")
    return r2

# Step 9: Polynomial fitting and other ML methods on expanded features
scaler = StandardScaler()

# 9.1 Polynomial Features (degree 2) on expanded
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

In [87]:
# Models
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'RandomForest': RandomForestRegressor(random_state=42)
}

# Hyperparameter grids for tuning
param_grids = {
    'Ridge': {'alpha': [0.1, 1.0, 10.0]},
    'RandomForest': {'n_estimators': [50, 100], 'max_depth': [None, 10]}
}

print("\nResults using Polynomial Features on Expanded Bands:")
for name, model in models.items():
    use_grid = name in param_grids
    if use_grid:
        grid = GridSearchCV(model, param_grids[name], cv=5, scoring='neg_mean_squared_error')
        grid.fit(X_train_poly_scaled, y_train)
        best_model = grid.best_estimator_
        print(f"Best params for {name}_poly: {grid.best_params_}")
    else:
        best_model = model
        best_model.fit(X_train_poly_scaled, y_train)
    
    pred = best_model.predict(X_test_poly_scaled)
    evaluate(y_test, pred, f'{name}_poly')


Results using Polynomial Features on Expanded Bands:
LinearRegression_poly - MSE: 3305.1044, R²: -1.6750
Best params for Ridge_poly: {'alpha': 10.0}
Ridge_poly - MSE: 699.7778, R²: 0.4336
Best params for RandomForest_poly: {'max_depth': None, 'n_estimators': 50}
RandomForest_poly - MSE: 646.1994, R²: 0.4770


In [ ]:

# 9.2 Other methods: Simple ML on expanded (without poly)
print("\nResults using ML on Expanded Bands (no poly):")
for name, model in models.items():
    use_grid = name in param_grids
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    if use_grid:
        grid = GridSearchCV(model, param_grids[name], cv=5, scoring='neg_mean_squared_error')
        grid.fit(X_train_scaled, y_train)
        best_model = grid.best_estimator_
        print(f"Best params for {name}: {grid.best_params_}")
    else:
        best_model = model
        best_model.fit(X_train_scaled, y_train)
    
    pred = best_model.predict(X_test_scaled)
    evaluate(y_test, pred, f'{name}')

In [83]:


# Step 10: Band Ratio Methods
# Generate all unique pairs of expanded columns for ratios
ratio_features = pd.DataFrame(index=df.index)
best_r2_linear = -np.inf
best_pair_linear = None
best_r2_quadratic = -np.inf
best_pair_quadratic = None

for col1, col2 in combinations(expanded_cols, 2):
    ratio = df[col1] / df[col2]
    ratio_name = f'{col1}/{col2}'
    ratio_features[ratio_name] = ratio

# Split ratio features
X_ratio_train, X_ratio_test, _, _ = train_test_split(ratio_features, y, test_size=0.2, random_state=42)

# For each ratio, fit linear and quadratic models
print("\nEvaluating Band Ratio Methods:")
for ratio_name in ratio_features.columns:
    # Linear: TSS ~ ratio
    X_r_train = X_ratio_train[[ratio_name]]
    X_r_test = X_ratio_test[[ratio_name]]
    lr = LinearRegression()
    lr.fit(X_r_train, y_train)
    pred_lr = lr.predict(X_r_test)
    r2_lr = r2_score(y_test, pred_lr)
    if r2_lr > best_r2_linear:
        best_r2_linear = r2_lr
        best_pair_linear = ratio_name
    
    # Quadratic: TSS ~ ratio^2 + ratio
    poly_ratio = PolynomialFeatures(degree=2, include_bias=False)
    X_r_train_poly = poly_ratio.fit_transform(X_r_train)
    X_r_test_poly = poly_ratio.transform(X_r_test)
    lr_poly = LinearRegression()
    lr_poly.fit(X_r_train_poly, y_train)
    pred_poly = lr_poly.predict(X_r_test_poly)
    r2_poly = r2_score(y_test, pred_poly)
    if r2_poly > best_r2_quadratic:
        best_r2_quadratic = r2_poly
        best_pair_quadratic = ratio_name

# Report best
print(f"\nBest Linear Band Ratio: {best_pair_linear} with R²: {best_r2_linear:.4f}")
print(f"Best Quadratic Band Ratio: {best_pair_quadratic} with R²: {best_r2_quadratic:.4f}")

# Step 11: Combined Approach - Add top ratios to expanded features and fit model
best_ratio = df[best_pair_quadratic.split('/')[0]] / df[best_pair_quadratic.split('/')[1]]
X_expanded_with_ratio = X_expanded.copy()
X_expanded_with_ratio['best_ratio'] = best_ratio

X_train_wr, X_test_wr, _, _ = train_test_split(X_expanded_with_ratio, y, test_size=0.2, random_state=42)

# Fit Ridge with poly
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('model', Ridge())
])

param_grid = {'model__alpha': [0.1, 1.0, 10.0]}
grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(X_train_wr, y_train)
best_model = grid.best_estimator_
print(f"Best params for Ridge with expanded + best ratio: {grid.best_params_}")

pred = best_model.predict(X_test_wr)
evaluate(y_test, pred, 'Ridge_Poly_With_Best_Ratio')

/tmp/ipykernel_1166/2713673858.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ratio_features[ratio_name] = ratio
/tmp/ipykernel_1166/2713673858.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ratio_features[ratio_name] = ratio
/tmp/ipykernel_1166/2713673858.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfra


Evaluating Band Ratio Methods:

Best Linear Band Ratio: Rrs_710/Rrs_713 with R²: 0.1051
Best Quadratic Band Ratio: Rrs_708/Rrs_713 with R²: 0.1283
Best params for Ridge with expanded + best ratio: {'model__alpha': 10.0}
Ridge_Poly_With_Best_Ratio - MSE: 593.3498, R²: 0.5198


0.5197649598851279